In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
image_names = get_png_names("../data/MEX/")[0:5]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

5 municipalities.


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

butler = geozoom_handler(attn_model, 
                         device, 
                         criterion, 
                         attn_optimizer, 
                         num_thresholds = 4,
                         reduction_percent = .65,
                         convergence_dims = (224, 224),
                         change_bounds = (-50, 50),
                         perc_change_thresh = .65, 
                         plot = False, 
                         v = False)

In [5]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [6]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [7]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  3
Num validation:  2


In [8]:
# image_names

In [9]:
butler.train_attn_model(train_dl, val_dl)

Epoch               Training Loss       Validation Loss     Threshold           % within bounds     


/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch: 0            2525.7508           1502.1957           3                   
Epoch: 1            2524.7222           1500.0909           3                   
Epoch: 2            2523.6133           1501.5779           3                   
Epoch: 3            2522.6939           1498.5865           3                   
Epoch: 4            2522.7098           1499.3737           2                   1.0                 
Epoch: 5            2523.1896           1497.9785           2                   
Epoch: 6            2522.5157           1294.0886           2                   
Epoch: 7            2521.7712           1363.1083           2                   
Epoch: 8            2520.7279           1237.533            1                   1.0                 
Epoch: 9            2521.317            1184.4402           1                   
Epoch: 10           2520.1539           1078.7366           1                   
Epoch: 11           2519.4605           1240.2811           1        

In [10]:
butler.train_fc_model(train_dl, val_dl, outside_estimator = 'mlp')

MLP MAE:  1903.903506036145


/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [11]:
def get_y(json_path, estimates):
    
    """
    Get a list of the y features corresponding to each image in image_names
    """
    
    m = open(json_path,)
    mig_data = json.load(m)
    m.close()
    muni_ids = [i for i in estimates]
    print(len(muni_ids), "municipalities.")  
    return [mig_data[i] for i in muni_ids]

In [11]:
x = np.array(list(butler.scale_estimates.values()))
y = np.array(get_y("../../pooling/data/migration_data.json", butler.scale_estimates.keys()))

15 municipalities.


In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

In [23]:
def mae(true, pred):
    diff = np.abs(true - pred)
    return np.mean(diff)

In [24]:
rfr = RandomForestRegressor()
rfr.fit(x, y)
mae(y, rfr.predict(x))

2712.458666666667

In [31]:
dtr = DecisionTreeRegressor(max_depth = 5)
dtr.fit(x, y)
mae(y, dtr.predict(x))

7.933333333333334

In [36]:
mlp = MLPRegressor()
mlp.fit(x, y)
mae(y, mlp.predict(x))

/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


5096.862278985803

In [38]:
knn = KNeighborsRegressor()
knn.fit(x, y)
mae(y, knn.predict(x))

4149.666666666666